In [1]:
import numpy as np; np.random.seed(1040941203) # For reproducibility (+82-10-4094-1203)
import pandas as pd

In [2]:
"""
    Step 3. Construct a classifier
"""
# Load manipulated data set
train_df = pd.read_pickle("train_set.csv")
test_df  = pd.read_pickle("test_set.csv")

In [3]:
# Check if the data frame is loaded normally
train_df.head()

,business_id,features,labels
0,3,"[0.17828523, 0.56636906, 1.39566, 0.34984797, ...",0 8
1,4,"[0.2780744, 0.43836534, 1.6742295, 0.36407334,...",1 2 4 5 6
2,5,"[0.52646494, 0.5042465, 2.1981008, 0.5013659, ...",1 2 4 5 6 7
3,6,"[0.2510131, 0.3978268, 2.6302445, 0.19584215, ...",0 1 6 8
4,7,"[0.15473817, 0.5289381, 1.585744, 0.45722356, ...",0 1 8


In [4]:
# Function to make labels in the data frame into a list (i.e. 0 8 => [0, 8])
def labels_to_list(labels): return list(map(int, labels.split()))

# Process train & test set into an array format
X_train = np.array([x for x in train_df['features']])
Y_train = np.array([labels_to_list(y) for y in train_df['labels']])
X_test = np.array([x for x in test_df['features']])

# Check shape of array-format train & test set 
print("X_train: ", X_train.shape)
print("Y_train: ", Y_train.shape)
print("X_test: ", X_test.shape)

X_train:  (1996, 2048)
Y_train:  (1996,)
X_test:  (10000, 2048)


In [6]:
# Load packages for splitting train & validation set, SVM classifier, 1-of-K encoder
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer

# Package for estimating a time taken
import time; t=time.time()

In [7]:
# Convert list of labels to follow 1-of-K coding scheme
one_of_K_encoder = MultiLabelBinarizer()
Y_train_ = one_of_K_encoder.fit_transform(Y_train)

# Split train set into 8:2 (train : validation)
random_state = np.random.RandomState(1040941203)
X_train_, X_test_, Y_train_, Y_test_ = train_test_split(X_train, Y_train_, test_size=0.2, random_state=random_state)

# Train the SVM classifier
classifier = OneVsRestClassifier(svm.SVC(kernel='rbf', probability=True))
classifier.fit(X_train_, Y_train_)

# Predict labels using the trained model
Y_predict = classifier.predict(X_test_)

# Show spent time
print("Time passed: ", "{0:.3f}".format(time.time() - t), "sec")

Time passed:  171.864 sec


In [8]:
# Show some predicted values
print("Samples of predicted labels (in 1-of-K coding scheme):\n", Y_predict[1:3])
print("\nSamples of corresponding predicted labels:\n", one_of_K_encoder.inverse_transform(Y_predict[1:3]))

Samples of predicted labels (in binary matrix):
 [[1 0 0 1 0 0 0 0 1]
 [0 0 0 1 0 1 1 0 1]]

Samples of predicted labels:
 [(0, 3, 8), (3, 5, 6, 8)]


In [9]:
# Construct a data frame to show ratio of each label in a predicted set
stat = pd.DataFrame(columns = ['label ' + str(i) for i in range(9)] + ['total_biz'], index = ['biz_count', 'biz_percentage'])

stat.loc['biz_count'] = np.append(np.sum(Y_predict, axis=0), len(Y_predict))
stat.loc['biz_percentage'] = stat.loc['biz_count'] * 100 / len(Y_predict) 

pd.options.display.float_format = '{:.0f}%'.format

# Show the statistics
stat

,label 0,label 1,label 2,label 3,label 4,label 5,label 6,label 7,label 8,total_biz
biz_count,109,186,200,156,97,247,291,106,247,400
biz_percentage,27%,46%,50%,39%,24%,62%,73%,26%,62%,100%


In [10]:
from sklearn.metrics import f1_score # For measuring F1 score metrics

# Show global F1 score & on-label F1 score
print("Overall F1 score: ", f1_score(Y_test_, Y_predict, average='micro')) 
print("F1 score of each label : ", f1_score(Y_test, y_ppredict, average=None))

F1 score:  0.8059084194977845
Individual Class F1 score:  [0.6446281  0.79283887 0.86699507 0.58536585 0.74509804 0.8990099
 0.91489362 0.69158879 0.86530612]


In [11]:
t = time.time()

# Convert list of labels to follow 1-of-K coding scheme
one_of_K_encoder = MultiLabelBinarizer()
Y_train_ = one_of_K_encoder.fit_transform(Y_train)

# Train the SVM classifier again with a full train set
random_state = np.random.RandomState(0)
classifier = OneVsRestClassifier(svm.SVC(kernel='rbf', probability=True))
classifier.fit(X_train, Y_train_)

Y_predict = classifier.predict(X_test)
Y_predict_label = one_of_K_encoder.inverse_transform(Y_predict)

print("Time passed: ", "{0:.1f}".format(time.time() - t), "sec")

Time passed:  486.2 sec


In [12]:
# Construct a data frame to show ratio of each label in a predicted set
stat = pd.DataFrame(columns=['label ' + str(i) for i in range(9)] + ['total_biz'], index = ['biz_count', 'biz_percentage'])

stat.loc['biz_count'] = np.append(np.sum(Y_predict, axis=0), len(Y_predict))
stat.loc['biz_percentage'] = stat.loc["biz_count"] * 100 / len(Y_predict)

pd.options.display.float_format = '{:.0f}%'.format

stat

,label 0,label 1,label 2,label 3,label 4,label 5,label 6,label 7,label 8,total_biz
biz_count,562,8086,8452,4278,891,9088,9408,970,6493,10000
biz_percentage,6%,81%,85%,43%,9%,91%,94%,10%,65%,100%


In [13]:
# Construct a data frame for submission (matching predicted label with business id in a test set)
final_df = pd.DataFrame(columns=['business_id','labels'])

for i in range(len(test_df)):
    biz = test_df.loc[i]['business_id']
    label = Y_predict_label[i]
    label = str(label)[1:-1].replace(",", " ")
    
    final_df.loc[i] = [str(biz), label]

# Write a submission file
with open("submission_Seokju_Hahn_SVM.csv",'w') as file:
    final_df.to_csv(file, index=False)